In [15]:
# Initialize Otter
import otter
grader = otter.Notebook("pcp4.ipynb")

ModuleNotFoundError: No module named 'otter'

# Programming Checkpoint 4: Exploring Temporal Energy Trends
## Deliverable Overview

In this assessment, You'll work with the OWID electricity dataset to create sophisticated visualizations that reveal patterns in global and regional electricity patterns, using diverse attributes and advanced styling techniques. Through a series of visualization tasks, you will analyze temporal changes in energy consumption, technology adoption, and resource independence.



## Dataset Description

This analysis uses the **Our World in Data (OWID) Energy Dataset**, which provides comprehensive global energy statistics from 1954 to 2024. The dataset includes metrics on energy production, consumption, and technology adoption across countries worldwide. Here is a description of the key columns you'll be working with:

| Column                        | Description                                                            | Unit                |
|-------------------------------|------------------------------------------------------------------------|---------------------|
| `country`                     | Country name                                                           | Geographic location |
| `year`                        | Year of observation                                                    | Date                |
| `nuclear_electricity`         | Electricity generation from nuclear                                    | TWh                 |
| `hydro_electricity`           | Electricity generated from hydropower                                  | TWh                 |
| `population`                  | Country population                                                     | Count               |
| `fossil_fuel_consumption`     | Primary energy consumption from fossil fuels                           | TWh                 |
| `coal_production`             | Coal production volume                                                 | TWh                 |
| `coal_consumption`            | Coal consumption volume                                                | TWh                 |
| `nuclear_share_energy`        | Share of primary energy consumption that comes from nuclear power      | %                   |
| `fossil_share_energy`         | Share of primary energy consumption that comes from fossil fuels       | %                   |
| `hydro_share_energy`          | Share of primary energy consumption that comes from hydropower         | %                   |
| `renewables_share_energy`     | Share of primary energy consumption that comes from other renewables   | %                   |
| `biofuel_consumption`         | Primary energy consumption from biofuels                               | TWh                 |
| `gas_consumption`             | Primary energy consumption from gas                                    | TWh                 |
| `oil_consumption`             | Primary energy consumption from oil                                    | TWh                 |
| `hydro_consumption`           | Primary energy consumption from hydropower                             | TWh                 |
| `other_renewable_consumption` | Primary energy consumption from other renewables                       | TWh                 |
| `wind_electricity`            | Electricity generation from wind power                                 | TWh                 |
| `solar_electricity`           | Electricity generation from solar power                                | TWh                 |
| `hydro_electricity`           | Electricity generation from hydropower                                 | TWh                 |

**Note:** TWh = Terawatt-hours (1 TWh = 1 billion kilowatt-hours)

<div class="alert alert-success" style="color: black; padding: 15px; border-radius: 8px; background-color: #d4edda;">
  <h3>Autograding System</h3>
  <p>This notebook uses <code>otter-grader</code> for immediate feedback. Run tests like:</p>
  <pre>grader.check("Task_A")</pre>
  <p>You'll receive feedback during the session, and additional hidden tests will run after submission on PrairieLearn.</p>
</div>

<div class="alert alert-info" style="color: black; padding: 15px; border-radius: 8px; background-color: #eaf4ff;">
  <h3>Instructions</h3>
  <ul>
    <li>Complete all tasks within the time limit</li>
    <li>Test your code as you go to ensure it runs without errors</li>
    <li>Focus on working solutions rather than perfect optimization</li>
    <li>Use pandas documentation if needed, but work efficiently</li>
  </ul>

  <h4>Submission Requirements:</h4>
  <ul>
    <li>Answer all questions and save your work</li>
    <li><strong>Before submitting:</strong> restart the kernel and rerun all cells (click the ▶▶ button)</li>
    <li>Click on the question title in the teal bar at the top to return to PrairieLearn, then click "Save and Grade"</li>
    <li>Don't change given variable names, move cells around, or include package installation code</li>
    <li>Submission may take 1-2 minutes to process</li>
  </ul>
</div>

## Setup

Let's import the libraries and load our energy dataset.

In [1]:
import pandas as pd
import altair as alt
import numpy as np

# Load the OWID energy dataset
energy_data = pd.read_csv('https://raw.githubusercontent.com/kemiolamudzengi/dsci-320-datasets/main/owid_dataset.csv')
energy_data['year'] = pd.to_datetime(energy_data['year'], format='%Y')

print(f"Dataset loaded successfully!")
print(f"Shape: {energy_data.shape}")
print(f"Years: {energy_data['year'].dt.year.min()} to {energy_data['year'].dt.year.max()}")
print(f"Countries: {energy_data['country'].nunique()}")

Dataset loaded successfully!
Shape: (1957, 132)
Years: 1964 to 2024
Countries: 33


<div style="background-color: #e8edf1ff; border-left:6px solid #d2d8deff; padding:15px; margin:15px 0;">

<h2>Data Preparation for your first task </h2>
<h5>Do not change anything in the cell below, it is us preparing the `americas_data` and `americas_clean` dataset for you. </h5>

</div>

In [5]:
# DO NOT CHANGE ANYTHING IN THIS CELL

americas_countries = ['United States', 'Canada', 'Mexico', 'Brazil', 'Colombia','Venezuela']

# Filter for Americas countries and year >= 2000
americas_data = energy_data.query('country in @americas_countries and year >= 2000').copy()

# Energy sources
energy_sources = [
    'biofuel_consumption',
    'gas_consumption',
    'oil_consumption',
    'hydro_consumption',
    'other_renewable_consumption'
]

# Select relevant data (columns: country, year, energy sources columns)
americas_energy = americas_data[['country', 'year'] + energy_sources].copy()

americas_energy['non_null_count'] = americas_energy[energy_sources].notna().sum(axis=1) 

americas_clean = americas_energy.query('non_null_count >= 3').copy() # SOLUTION

for source in energy_sources:
    americas_clean[source] = americas_clean[source].fillna(0) # SOLUTION


<div style="background-color:#fff3cd; border-left:6px solid #ffecb5; padding:15px; margin:15px 0;">
  <h2>DATA TASK: Americas Energy Consumption DataFrame</h2>
    
  <p><strong>Given:</strong> The <code>energy_data</code> DataFrame, which contains global energy data.</p>
  <p><strong>Goal:</strong> Prepare a dataset of energy consumption for selected Americas countries after the year 2000. We have done the first few steps for you</p>
  
  <ul>
    <li> We have created the <code>americas_data</code> and the <code>americas_clean</code> dataframes. </li>
    <li> We have also removed rows with too many missing values, and replace the remaining missing values with 0s.
    </li>
  </ul>
    <h3>Your task is to do the following: </h3>
    <ol>
      <b>1. Create <code>americas_long </code> :</b>
      Reshape the data from wide to long format using <code>melt</code> for the columns in <code>energy_sources</code>.  The resulting columns should be:
      <ul>
        <li><code>energy_source</code> – the type of energy source</li>
        <li><code>consumption_twh</code> – the consumption values</li>
      </ul>
        <br>
        You need to fill in the values for the melt function
        <pre><code class="language-python">
        americas_long = pd.melt(
            americas_clean,
            id_vars=[...],
            value_vars=[...],
            var_name='...',
            value_name='...'
        )
      </code></pre>
    </ol> 
   <ul>
    <li>
      <b>2. Update <code>americas_long</code>:</b>
      Clean up the <code>energy_source</code> names by removing <code>"_consumption"</code> and capitalizing them.</li> <li>
      <b>3. Create <code>americas_consumption</code>:</b>
      Remove rows with zero <code>consumption_twh</code>.
       </li>
    </ul>
</div>


_Points:_ 15

In [8]:
# TODO: Reshape data for stacked area visualization
americas_long = pd.melt(
    americas_clean,
    id_vars=['country', 'year'],
    value_vars = energy_sources,
    var_name = 'energy_source',
    value_name = 'consumption_twh'
)

# TODO: Clean source names
americas_long['energy_source'] = americas_long['energy_source'].map(
    {'biofuel_consumption' : 'Biofuel',
    'gas_consumption' : 'Gas',
    'oil_consumption' : 'Oil',
    'hydro_consumption' : 'Hydro',
    'other_renewable_consumption' : 'Other'
    }
)

# TODO: Remove zero consumption
americas_consumption = americas_long.query('consumption_twh > 0').copy()

<div style="background-color: #d2e6f8ff; border-left:6px solid #9dcffaff; padding:15px; margin:15px 0;">

<h2>VIZ TASK: Americas Energy Mix Stacked Trellis Chart</h2>
<h5>Create a stacked trellis area chart (using <code>americas_long</code> showing energy consumption by source across multiple countries.</h5>
<h4>Chart Specifications:</h4>
<ul>
  <li><b>Mark:</b> <code>area</code></li>
  <li><b>X channel:</b> <code>year:T</code>, title = "Year".</li>
  <li><b>Y channel:</b> <code>consumption_twh:Q</code>, title = "Energy Consumption (TWh)", with stacking set to zero.</li>
  <li><b>Color channel:</b> <code>energy_source:N</code>.</li>
  <li><b>Tooltip:</b> Show <code>country</code>, <code>year</code>, <code>energy_source</code>, and <code>consumption_twh</code>. Format <code>consumption_twh</code> values to 2 decimal places <code>".2f"</code>).</li>
  <li><b>Facet:</b> 3-column faceting by <code>country</code></li>
</ul>

<h4>Styling Specifications:</h4>
<ul>
  <li><b>Chart Properties:</b> Width = 200px, Height = 150px.</li>
  <li><b>Mark Styling:</b> strokeWidth = 3.</li>
  <li><b>Title:</b> <i>"Americas Energy Mix Evolution: Biofuels, Gas, Oil, Hydro & Other Renewables"</i>.</li>
  <li><b>Legend:</b> Title = "Energy Source".</li>
  <li><b>Color Scheme:</b> Use custom color range <code>range=['#E67E22', '#3498DB', '#8B4513', '#27AE60', '#9B59B6']</code>.</li>
  <li><b>Scale Resolution:</b> Y channel scales set to independent (each subplot has its own y-scale).</li>
</ul>

</div>


_Points:_ 25

In [27]:
# Create stacked trellis area chart
americas_stacked_trellis_chart = alt.Chart(americas_consumption).mark_area(strokeWidth = 3).encode(
    alt.X('year:T', title = 'Year'),
    alt.Y('consumption_twh:Q', title = 'Energy Consumption (TWh)', stack = 'zero'),
    alt.Color('energy_source:N', title = "Energy Source", scale = alt.Scale(range=['#E67E22', '#3498DB', '#8B4513', '#27AE60', '#9B59B6'])),
    tooltip = ['country', 'year', 'energy_source', 
               alt.Tooltip('consumption_twh', format = '.2f')],
    facet=alt.Facet('country:N', title=None, columns = 4),  # <- key change
).properties(
    width = 200,
    height = 150,
    title = "Americas Energy Mix Evolution: Biofuels, Gas, Oil, Hydro & Other Renewables"
).resolve_scale(
    y = 'independent'
)

# Show the plot
americas_stacked_trellis_chart

alt.Chart(...)

In [14]:
grader.check("Task_VIZ_B")

NameError: name 'grader' is not defined

<div style="background-color: #e8edf1ff; border-left:6px solid #d2d8deff; padding:15px; margin:15px 0;">

<h2>Data Preparation: Coal-Producing Countries</h2>
<h5>For the following tasks, we will use data from major coal-producing countries. The data has been filtered and prepared for you. </h5>

</div>

In [33]:
# Define coal-producing countries
coal_countries = ['Russia', 'Australia', 'Indonesia', 'South Africa', 'Colombia']

# Filter dataset for coal countries and select relevant columns
coal_data = energy_data.query('country in @coal_countries')[
    ['country', 'region', 'year', 'coal_production', 'coal_consumption', 'nuclear_share_energy',
     'population', 'fossil_share_energy', 'hydro_share_energy', 'renewables_share_energy']
].dropna()

# Prepare coal independence data
coal_independence = coal_data.dropna(subset=['coal_production', 'coal_consumption']).copy()

# Calculate independence ratio
coal_independence['coal_independence'] = coal_independence['coal_production'] / coal_independence['coal_consumption']

# Create decade groups
coal_independence['year'] = coal_independence['year'].dt.year
coal_independence['decade'] = (coal_independence['year'] // 10) * 10
coal_independence['decade_label'] = coal_independence['decade'].astype(int).astype(str) + 's'

# Aggregate by country and decade
heatmap_data = coal_independence.groupby(['country', 'decade_label'])['coal_independence'].mean().reset_index()

print(f"Coal data shape: {heatmap_data.shape}")
print(f"Countries: {sorted(heatmap_data['country'].unique())}")
print(f"Year range: {heatmap_data['decade_label'].min()}-{heatmap_data['decade_label'].max()}")
print(f"Sample data:")
print(heatmap_data.head())

Coal data shape: (26, 3)
Countries: ['Colombia', 'Indonesia', 'Russia', 'South Africa']
Year range: 1960s-2020s
Sample data:
    country decade_label  coal_independence
0  Colombia        1960s           1.015402
1  Colombia        1970s           1.134483
2  Colombia        1980s           2.046753
3  Colombia        1990s           6.369829
4  Colombia        2000s          14.594415


<div style="background-color: #d2e6f8ff; border-left:6px solid #9dcffaff; padding:15px; margin:15px 0;">

<h2>VIZ TASK: Coal Independence by Country and Decade</h2>


<h4>Chart Specifications:</h4>
<ul>
  <li><b>Mark:</b> <code>rect</code></li>
  <li><b>X channel:</b> <code>decade_label:O</code>, title = "Decade".</li>
  <li><b>Y channel:</b> <code>country:N</code>, title = "Country", sorted by mean <code>coal_independence</code> (ascending).</li>
  <li><b>Color channel:</b> <code>coal_independence:Q</code>, title='Coal Independence Ratio'.</li>
  <li><b>Tooltip:</b> Show <code>country</code>, <code>year</code>, and <code>coal_independence</code>. Format <code>coal_independence</code> to 2 decimal places (<code>".2f"</code>).</li>
</ul>

<h4>Styling Specifications:</h4>
<ul>
  <li><b>Chart Properties:</b> Width = 400px, Height = 300px.</li>
  <li><b>Title:</b> <i>"Coal Independence by Country and Decade: Green = Net Exporter, Purple = Net Importer"</i>.</li>
  <li><b>Legend:</b> Title = "Coal Independence Ratio".</li>
  <li><b>Color Scheme:</b> Use <code>purplegreen</code> (Purple = lower values, Green = higher values) with domain [0, 5].</li>
</ul>

</div>


_Points:_ 22

In [43]:
# Create heatmap
coal_heatmap = alt.Chart(coal_independence).mark_rect().encode(
    alt.X('decade_label:O', title = 'Decade'),
    alt.Y('country:N', title = 'Country', sort={
            'field': 'coal_independence',
            'op': 'mean',
            'order': 'ascending'
        })
    alt.Color('coal_independence:Q', title = 'Coal Independence Ratio', scale = alt.Scale(domain = [0,5], scheme = 'purplegreen')),
    tooltip = ['country', 'year:T',
               alt.Tooltip('coal_independence', format = '.2f')]
).properties(
    title = "Coal Independence by Country and Decade: Green = Net Exporter, Purple = Net Importer",
    width = 400,
    height = 300,
)
             

# Show the plot
coal_heatmap

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3579617748.py, line 4)

In [ ]:
grader.check("Task_VIZ_C")

<div style="background-color: #d2e6f8ff; border-left:6px solid #9dcffaff; padding:15px; margin:15px 0;">

<h2>VIZ TASK: Coal Independence Lollipop Chart</h2>

<p><strong>Exploratory Question:</strong>
<em>How has coal independence changed over decades for Colombia?</em></p>

<p> Create a horizontal lollipop chart showing coal independence for each decade.</p>
    Don't fret. You haven't seen this viz before, but you have worked with layers on the Tutorial and so we will do something similar. We will start out with a base layer and then add the sticks and circle layers on top to create the lollipop effect
    

<h3>Chart Specification:</h3>

<h4>STEP 1: Create a base chart using the <code>colombia_data</code>dataset. For this chart </h4>
<ul>
    <li>Encode <code>decade_label</code> on the <strong>y channel</strong> as ordinal.</li>
    <li>Encode <code>coal_independence</code> on the <strong>x channel</strong> as quantitative.</li>
    <li>Set y-axis title to <code>"Decade"</code>.</li>
    <li>For the y axis, remove the ticks but keep the labels (<code>ticks = False</code>, <code>labels = True</code>)</li>
    <li>NOTE: This chart has no mark but don't worry it will be addressed later </li>
</ul>

<h4>STEP 2: Use mark_rule to draw the "sticks" from 0 to the coal independence value for each decade. </h4>
<ul>
    <li>Startwith the mark<code>mark_rule</code></li>    
    <li>Encode <code>coal_independence</code> on the <strong>x channel</strong></li>
    <li>Set x-axis title to <code>"Coal Independence Ratio"</code>.</li>
    <li>For the x axis, remove the ticks but keep the labels (<code>ticks = False</code>, <code>labels = True</code>)</li>
    <li>Encode <code>alt.value(0)</code> on the <strong>x2 channel</strong> as quantitative.</li>
    
</ul>

<h4>STEP 3:Use mark_circle to draw the "heads" of the lollipops at the coal independence value. </h4>
<ul>
      <li>Encode <code>coal_independence</code> on the <strong>x channel</strong></li>  
      <li>Circle color: <code>red</code>.</li>
      <li>Circle size: 120 pixels.</li>
</ul>

<h3>Additional Styling Specifications:</h3>
<ul>
    <li>Remove axis ticks but keep labels readable.</li>
    <li>Width: 300px, Height: 150px.</li>
    <li>Chart title: <em>"Coal Independence by Decade (Colombia)"</em>.</li>
</ul>




_Points:_ 18

In [50]:
## DATA PREPARATION DO NOT EDIT THIS CELL

colombia_data = heatmap_data.query('country == "Colombia"').copy()


In [66]:

# Base chart
base = alt.Chart(colombia_data).encode(
    alt.Y('decade_label:O', title = 'Decade', axis = alt.Axis(
        ticks = False, labels = True)
         ),
    alt.X('coal_independence:Q')
)

# Lollipop "stick"
sticks = base.mark_rule().encode(
    alt.X('coal_independence:Q',title = "Coal Independence Ratio", axis = alt.Axis(
        ticks = False, labels = True)
         ),
    x2 = alt.value(10)
)

# Lollipop "head"
circles = base.mark_circle(
    fill = 'red',
    size = 120
).encode(
    alt.X('coal_independence:Q')
)

# Combine and set the global properties of width, height and title.

lollipop = alt.layer(circles, sticks)


# SHOW CHART
lollipop



alt.LayerChart(...)

In [ ]:
grader.check("Task_VIZ_D")

In [68]:
# DO NOT CHANGE ANYTHING IN THIS CELL
# Data preparation for next task
energy_data = energy_data[~energy_data['country'].isin(["World", "Africa", "Asia", "Europe", "North America", "South America", "South Africa"])]
energy_data.head(5)

,country,year,iso_code,entity_type,region,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
122,Bahrain,1964-01-01,BHR,country,Asia,193779.0,9.095145e+08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,Bahrain,1965-01-01,BHR,country,Asia,200837.0,9.679900e+08,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,Bahrain,1966-01-01,BHR,country,Asia,207503.0,1.029362e+09,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,Bahrain,1967-01-01,BHR,country,Asia,214141.0,1.096433e+09,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,Bahrain,1968-01-01,BHR,country,Asia,219299.0,1.166028e+09,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div style="background-color:#fff3cd; border-left:6px solid #ffecb5; padding:10px; margin:10px 0;">
<h2> DATA TASK - Creating new Dataframes</h2>
Work through the following steps with <code>energy_data</code>.
 <br> At each step, create a new DataFrame/list  with the specified name (wrapped in <code>...</code>):

  <ol>
  <li>
    <b>Create <code>large_economies_2020</code>:</b>
    Filter <code>energy_data</code> to keep only rows where the year is 2020, the <code>gdp</code> is greater than 2000 billion (2000000000000 dollars), and drop rows with missing <code>gdp</code>.
  </li>
    <br>
  <li>
    <b>Create <code>large_economies</code>:</b>
    From <code>energy_data</code>, keep all data (i.e., rows) for the countries that appear in <code>large_economies_2020</code>.
  </li>
      <br>

  <li>
    <b>Create <code>nuclear_leaders_list</code>:</b>
    Identify the countries (from <code>energy_data</code>) that produced more than 100 TWh of <code>nuclear_electricity</code> in any year. Store the unique country names in <code>nuclear_leaders_list</code>.
  </li>
      <br>

  <li>
    <b>Create <code>nuclear_leaders</code>:</b>
    From <code>large_economies</code>, keep only rows where <code>country</code> is in <code>nuclear_leaders_list</code>.
  </li>
      <br>

  <li>
    <b>Create <code>nuclear_recent</code>:</b>
    From <code>nuclear_leaders</code>, keep only rows with years between 2015 and 2020 (inclusive).
  </li>
</ol>


    
 Hint: Break it down step by step—each filter builds on the last.

  <pre><code class="language-python">
# Step 1: large_economies_2020
large_economies_2020 = energy_data[ ... ]

# Step 2: large_economies
large_economies = energy_data[ ... ]

# Step 3: nuclear_leaders_list
nuclear_leaders_list = energy_data[ ... ]

# Step 4: nuclear_leaders
nuclear_leaders = large_economies[ ... ]

# Step 5: nuclear_recent
nuclear_recent = nuclear_leaders[ ... ]
  </code></pre>
</div>


_Points:_ 20

In [71]:
# Step 1: large_economies_2020
large_economies_2020 = (
    energy_data
        .dropna(subset=['gdp'])
        .query("year == 2020 and gdp > 2000000000000")
)

# Step 2: large_economies
large_economies = energy_data.query(
    "country in @large_economies_2020.country"
)

# Step 3: nuclear_leaders_list
nuclear_leaders_list = (
    energy_data
        .query("nuclear_electricity > 100")
        ['country']
        .unique()
)

# Step 4: nuclear_leaders
nuclear_leaders = large_economies.query(
    "country in @nuclear_leaders_list"
)

# Step 5: nuclear_recent
nuclear_recent = (
    nuclear_leaders
        .query("2015 <= year <= 2020")
)
nuclear_recent

,country,year,iso_code,entity_type,region,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
417,China,2015-01-01,CHN,country,Asia,1.396134e+09,1.830122e+13,-26.129,-9.656,19.553,...,0.679,0.286,16.167,63.331,469.852,132.931,185.59,336.538,3.192,1.346
418,China,2016-01-01,CHN,country,Asia,1.404053e+09,1.954571e+13,1.889,0.516,19.810,...,1.085,0.477,29.780,135.325,605.177,171.546,240.86,431.022,3.927,1.728
419,China,2017-01-01,CHN,country,Asia,1.412355e+09,2.089436e+13,-6.755,-1.879,18.364,...,1.784,0.810,26.463,156.323,761.500,215.668,304.60,539.171,4.612,2.093
420,China,2018-01-01,CHN,country,Asia,1.419009e+09,2.229429e+13,29.818,7.733,23.727,...,2.469,1.155,20.092,148.450,909.950,257.786,365.80,641.258,5.105,2.388
421,China,2019-01-01,CHN,country,Asia,1.423520e+09,2.363194e+13,7.079,2.383,25.327,...,2.985,1.391,10.798,93.268,1003.218,284.717,405.30,704.744,5.402,2.517
422,China,2020-01-01,CHN,country,Asia,1.426106e+09,2.415184e+13,-23.742,-8.560,19.279,...,3.356,1.567,15.100,148.634,1151.852,327.115,466.50,807.690,5.997,2.800
765,France,2015-01-01,FRA,country,Europe,6.491629e+07,2.464954e+12,0.873,0.283,504.463,...,1.350,0.660,23.765,10.360,55.136,329.963,21.42,849.334,3.731,1.979
766,France,2016-01-01,FRA,country,Europe,6.508680e+07,2.491956e+12,0.605,0.198,506.186,...,1.550,0.752,-0.197,-0.524,54.612,328.484,21.38,839.065,3.826,2.001
767,France,2017-01-01,FRA,country,Europe,6.528473e+07,2.549058e+12,0.781,0.257,508.595,...,1.723,0.836,15.184,7.978,62.590,376.964,24.61,958.719,4.423,2.300
768,France,2018-01-01,FRA,country,Europe,6.551948e+07,2.596599e+12,-1.137,-0.378,501.010,...,1.877,0.933,16.245,9.805,72.395,436.511,28.60,1104.940,4.966,2.630


<div class="alert alert-success" style="color: black; padding: 15px; border-radius: 8px; background-color: #d4edda;">
<h3>Programming Pulse Check Complete!</h3>
<h4>Final Submission Steps:</h4>
<ul>
<li><strong>Restart and run all cells:</strong> Click the ▶▶ button or go to <code>Kernel → Restart Kernel and Run All Cells...</code> in the menu to ensure there are no errors</li>
<li><strong>Save your file:</strong> Make sure your work is saved</li>
<li><strong>Submit your assessment:</strong> Return to the main PL assessment page for the Quiz and submit your entire assessment</li>
</ul>
</div>